In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import re
import networkx as nx
import nltk
nltk.download(['punkt','stopwords'])
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
# from nltk.corpus import stopwords
# from nltk import bigrams

# import warnings
# warnings.filterwarnings("ignore")

# sns.set(font_scale=1.5)
# sns.set_style("whitegrid")
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# analyzer = SentimentIntensityAnalyzer()%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

[nltk_data] Downloading package punkt to /Users/ellie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ellie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
dataset_df = pd.read_csv('dataset_to_train.csv')

dataset_df = dataset_df.drop_duplicates()
dataset_df

,Unnamed: 0,tweet,class
0,0,@kooawr watching amas :)),0
1,1,still can't wrap my head around why this zoo c...,0
2,2,SAVE YOUR TEARS FOR ANOTHER DAY,0
3,3,Good day!!! #surfing #somewhere #goodwave #en...,0
4,4,@poppingin50 have a nice day yh,0
...,...,...,...
1395,95,But in the meantime...? Our basic needs are be...,1
1396,96,@voltruto kaya pala trending ka always ning HH...,1
1397,97,the more illegal the better i always say its hot,1
1398,98,@simplysarris i feel bad even if i mute someon...,1


In [19]:
(dataset_df.isnull().any().any())
## No null values in our dataset- can ignore removing

False

In [20]:
dataset_df['clean_tweet'] = dataset_df['tweet'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))

In [21]:
dataset_df

,Unnamed: 0,tweet,class,clean_tweet
0,0,@kooawr watching amas :)),0,@kooawr watching amas :))
1,1,still can't wrap my head around why this zoo c...,0,still can't wrap head around zoo close I live ...
2,2,SAVE YOUR TEARS FOR ANOTHER DAY,0,SAVE YOUR TEARS FOR ANOTHER DAY
3,3,Good day!!! #surfing #somewhere #goodwave #en...,0,Good day!!! #surfing #somewhere #goodwave #enj...
4,4,@poppingin50 have a nice day yh,0,@poppingin50 nice day yh
...,...,...,...,...
1395,95,But in the meantime...? Our basic needs are be...,1,But meantime...? Our basic needs MET. We're st...
1396,96,@voltruto kaya pala trending ka always ning HH...,1,@voltruto kaya pala trending ka always ning HH...
1397,97,the more illegal the better i always say its hot,1,illegal better always say hot
1398,98,@simplysarris i feel bad even if i mute someon...,1,@simplysarris feel bad even mute someone 🙃 alw...


In [23]:
dataset_df['text'] = dataset_df['clean_tweet']
dataset_df['text']

0                               @kooawr watching amas :))
1       still can't wrap head around zoo close I live ...
2                         SAVE YOUR TEARS FOR ANOTHER DAY
3       Good day!!! #surfing #somewhere #goodwave #enj...
4                                @poppingin50 nice day yh
                              ...                        
1395    But meantime...? Our basic needs MET. We're st...
1396    @voltruto kaya pala trending ka always ning HH...
1397                        illegal better always say hot
1398    @simplysarris feel bad even mute someone 🙃 alw...
1399    @dyutka LMFAO SAME OMG used go one jamaica itt...
Name: text, Length: 1400, dtype: object

In [25]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())
all_tweets_no_urls = [remove_url(tweet) for tweet in dataset_df['text']]
all_tweets_no_urls[:5]

['kooawr watching amas',
 'still cant wrap head around zoo close I live phantom cave dude playing him hed appear certain times day went get stamp hand ring',
 'SAVE YOUR TEARS FOR ANOTHER DAY',
 'Good day surfing somewhere goodwave enjoy Timmys Cafe',
 'poppingin50 nice day yh']

In [39]:
#lower_case = [word.lower() for word in df_new['text']]
dataset_df["clean_tweet"]=dataset_df["clean_tweet"].str.lower()
dataset_df['clean_tweet'] = dataset_df['clean_tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [40]:
dataset_df

,Unnamed: 0,tweet,class,clean_tweet,text
0,0,@kooawr watching amas :)),0,@kooawr watching amas :)),@kooawr watching amas :))
1,1,still can't wrap my head around why this zoo c...,0,still can't wrap head around zoo close i live ...,still can't wrap head around zoo close I live ...
2,2,SAVE YOUR TEARS FOR ANOTHER DAY,0,save your tears for another day,SAVE YOUR TEARS FOR ANOTHER DAY
3,3,Good day!!! #surfing #somewhere #goodwave #en...,0,good day!!! #surfing #somewhere #goodwave #enj...,Good day!!! #surfing #somewhere #goodwave #enj...
4,4,@poppingin50 have a nice day yh,0,@poppingin50 nice day yh,@poppingin50 nice day yh
...,...,...,...,...,...
1395,95,But in the meantime...? Our basic needs are be...,1,but meantime...? our basic needs met. we're st...,But meantime...? Our basic needs MET. We're st...
1396,96,@voltruto kaya pala trending ka always ning HH...,1,@voltruto kaya pala trending ka always ning hh...,@voltruto kaya pala trending ka always ning HH...
1397,97,the more illegal the better i always say its hot,1,illegal better always say hot,illegal better always say hot
1398,98,@simplysarris i feel bad even if i mute someon...,1,@simplysarris feel bad even mute someone 🙃 alw...,@simplysarris feel bad even mute someone 🙃 alw...


In [42]:
dataset_df.to_csv(r'/Users/ellie/Desktop/dataset_to_train.csv')